# This script calculates the mean and standard deviation for the FER2013 dataset images.

In [1]:

import kagglehub
import numpy as np

import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Download and Transform

In [2]:
msambare_fer2013_path = kagglehub.dataset_download('msambare/fer2013')

print('Import of FER2013 complete')

TRAIN_IMG_FOLDER_PATH = "../kaggle/input/fer2013/train"
VALID_IMG_FOLDER_PATH = "../kaggle/input/fer2013/test"

DEVICE = 'cuda'

BATCH_SIZE = 32
EPOCHS = 200
LR = 0.001


DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class AlbumentationsTransform: # Änderung
    def __init__(self, aug):
        self.aug = aug

    def __call__(self, img):
        img = np.array(img)  # PIL → NumPy
        augmented = self.aug(image=img)
        return augmented['image']


# Resize + ToTensor only (no Normalize!)
simple_transform = AlbumentationsTransform(
    A.Compose([
        A.Resize(64, 64),
        ToTensorV2()
    ])
)

# Load trainset without normalization
trainset_for_stats = ImageFolder(TRAIN_IMG_FOLDER_PATH, transform=simple_transform)

Import of FER2013 complete


# Calculate Mean and Std

In [3]:

from torch.utils.data import DataLoader
from tqdm import tqdm

loader = DataLoader(trainset_for_stats, batch_size=128, shuffle=False, num_workers=2)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in tqdm(loader):
    data = data.float() / 255.0  # Convert from uint8 [0,255] to float [0,1]
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f"Mean: {mean}")
print(f"Std: {std}")

class AlbumentationsTransform: # Änderung
    def __init__(self, aug):
        self.aug = aug

    def __call__(self, img):
        img = np.array(img)  # PIL → NumPy
        augmented = self.aug(image=img)
        return augmented['image']

# Resize + ToTensor only (no Normalize!)
simple_transform = AlbumentationsTransform(
    A.Compose([
        A.Resize(64, 64),
        ToTensorV2()
    ])
)

# Load trainset without normalization
trainset_for_stats = ImageFolder(TRAIN_IMG_FOLDER_PATH, transform=simple_transform)

from torch.utils.data import DataLoader
from tqdm import tqdm

loader = DataLoader(trainset_for_stats, batch_size=128, shuffle=False, num_workers=2)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in tqdm(loader):
    data = data.float() / 255.0  # Convert from uint8 [0,255] to float [0,1]
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f"Mean: {mean}")
print(f"Std: {std}")


100%|██████████| 225/225 [01:30<00:00,  2.50it/s]


Mean: tensor([0.5073, 0.5073, 0.5073])
Std: tensor([0.2061, 0.2061, 0.2061])


100%|██████████| 225/225 [00:21<00:00, 10.34it/s]

Mean: tensor([0.5073, 0.5073, 0.5073])
Std: tensor([0.2061, 0.2061, 0.2061])
